**Importando bibliotecas**

In [107]:
import pandas as pd
import folium

   **Criando um função de  mapa de calor com os casos de Dengue em subdistritos,
    utilizando dados de um arquivo Excel e um arquivo GeoJSON.**

In [108]:
def criar_mapa_dengue(nome_arquivo_geojson, nome_arquivo_excel):
    # Carregar dados de casos de Dengue
    casos = pd.read_excel(nome_arquivo_excel)
    casos = casos.dropna(subset=["subdistrito"])
    casos["casos"] = casos[[2019, 2020, 2021, 2022, 2023]].sum(axis=1)
    
    # Criar o mapa
    mapa = folium.Map([-15.7801, -47.9292], zoom_start=9)
    
    # Adicionar o mapa de coropléticos
    folium.Choropleth(
        geo_data=nome_arquivo_geojson,
        data=casos,
        columns=["subdistrito", "casos"],
        key_on="feature.properties.id",
        fill_color="GnBu",
        fill_opacity=0.9,
        line_opacity=0.5,
        legend_name="DENGUE",
        nan_fill_color="white",
        name="Casos de Dengue"
    ).add_to(mapa)
    
    # Estilo para os elementos não destacados
    estilo = lambda x: {
        "fillColor": "white",
        "color": "black",
        "fillOpacity": 0.001,
        "weight": 0.001
    }

    # Estilo para os elementos destacados
    estilo_destaque = lambda x: {
        "fillColor": "darkblue",
        "color": "black",
        "fillOpacity": 0.5,
        "weight": 1
    }

    # Adicionando o GeoJson para Cidades
    highlight = folium.features.GeoJson(
        data=nome_arquivo_geojson,
        style_function=estilo,
        highlight_function=estilo_destaque,
        name="Cidades"
    )

    # Adicionando caixa de texto para Cidades
    folium.features.GeoJsonTooltip(
        fields=["name", "id"],
        aliases=["Subdistrito:", "Casos"],
        labels=True,
        style="background-color: white; color: black; font-family: arial; font-size: 16px; padding: 10px;"
    ).add_to(highlight)

    # Adicionando o destaque ao mapa
    mapa.add_child(highlight)

    # Adicionando o controle de camadas
    folium.LayerControl().add_to(mapa)

    # Retornar o mapa
    return mapa

In [109]:
df = pd.read_excel('casos-dengue.xlsx')

In [110]:
df = df.iloc[:, :-1]

In [111]:
linhas_a_pagar = [0, 7, 16, 21, 26, 29, 35, 39]
linhas_cidades = df.drop(index=linhas_a_pagar)

In [112]:
linhas_cidades

,Região de Saúde,2019,2020,2021,2022,2023
1,. Cruzeiro,278,359,92,570,199
2,. Lago Norte,422,512,299,702,283
3,. Lago Sul,300,460,127,529,223
4,. Plano Piloto,1658,2008,607,1896,1047
5,. Sudoeste Octogonal,220,179,91,239,146
6,. Varjão,662,126,42,203,84
8,. Candangolândia,365,261,43,263,101
9,. Estrutural,563,237,188,671,353
10,. Guará,1828,2887,481,2296,745
11,. Núcleo Bandeirante,591,251,88,300,143


**Baixa um novo excel somente com as cidades**

In [113]:
linhas_cidades.to_excel('resultado_analise_cidade.xlsx', index=False)

**Cria um mapa de calor que representa a incidência de casos de Dengue nos subdistritos, utilizando dados de um arquivo Excel e geometrias de um arquivo GeoJSON.**

In [117]:

nome_arquivo_geojson = "mapa - Copia.json"
nome_arquivo_excel = "subdistritos.xlsx"
mapa_dengue = criar_mapa_dengue(nome_arquivo_geojson, nome_arquivo_excel)
mapa_dengue
